### 기본 라이브러리 설정

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from collections import Counter
import konlpy
from konlpy.tag import Mecab
import time
import json
from random import shuffle
from tqdm import tqdm
tqdm.pandas()
import gensim 
from gensim.models import Word2Vec, FastText, Doc2Vec
import fasttext
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 목차
### 1. 크롤링
### 2. 데이터 전처리
### 3. 단어 벡터 만들기
- 1) tokenize 완료된 bills를 json으로 저장
- 2) 학습한 corpus 만들기
- 3) word2vec 학습시켜서 단어 벡터 만들기 
- 4) 단어와 벡터를 dictionary 형태로 묶기  

###  4. 법안 벡터 만들기
- 1) 단어벡터를 합쳐서 법안벡터 만들기  

### 5. 국회의원 벡터 만들기
### 6. 의원, 단어, 법안 시각화하기
- 1) 의원 시각화
- 2) 단어 시각화
- 3) 법안 시각화

### 7. 워드 클라우드 만들어보기

### 8. 사용자 Input 받아서 국회의원 추천하기
- 1) 첫 번째 시연: 20대 취업준비중인 청년
- 2) 두 번째 시연: 장애인 자녀를 키우고 있는 부모


# 1. 크롤링

In [ ]:
# 기본 세팅 
from selenium import webdriver
path = r"chromedriver.exe"
driver = webdriver.Chrome(path)

In [ ]:
### 열려라 국회 의원정보 & 사진 크롤링
# 열러라 국회 의원 접속
driver.get('http://watch.peoplepower21.org/?act=&mid=AssemblyMembers&vid=&mode=search&name=&party=&region=&sangim=&gender=&age=&elect_num=')

start_time = time.time()
list_name = []
list_party = []
list_region = []
list_elected_times = []
list_committee = []
list_education = []
list_career = []
list_tel = []
list_email = []
list_bill_count = []
list_committee_attend = []
list_meeting_attend = []
list_property = []
for page in range(1,11):
    if page != 1:
        driver.get('http://watch.peoplepower21.org/?mid=AssemblyMembers&mode=search&party=&region=&sangim=&gender=&elect_num=&page=' + str(page))
    for person in range(1, 31):
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[' + str(person) + ']/div').click()
        
        full_name = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > h1')
        party = driver.find_elements_by_css_selector('div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(1) > td:nth-child(2)')
        region = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(2) > td:nth-child(2) > a')
        elected_times = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(3) > td:nth-child(2)')
        committee = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(4) > td:nth-child(2) > a')
        education = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(5) > td:nth-child(2)')
        career = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(6) > td:nth-child(2)')
        tel = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(7) > td:nth-child(2)')
        email = driver.find_elements_by_css_selector('#content > div > div > div > div.panel.panel-default > div.panel-body > div > div.col-md-9.col-lg-9 > table > tbody > tr:nth-child(8) > td:nth-child(2) > a')
        bill_count = driver.find_elements_by_css_selector('#collapse1 > div > h3 > span')
        committee_attend = driver.find_elements_by_css_selector('#collapse2 > div > span > span > span > span')
        meeting_attend = driver.find_elements_by_css_selector('#collapse3 > div > p:nth-child(3) > span > span > span > span')
        property_ = driver.find_elements_by_css_selector('#collapse5 > div > table > tbody > tr.info')
        if len(committee) != 0:
            list_committee.append(committee[0].text)
        else:
            list_committee.append(np.nan)
        if len(committee_attend) != 0:
            list_committee_attend.append(committee_attend[0].text)
        else:
            list_committee_attend.append(np.nan)
        if len(property_) != 0:
            list_property.append(property_[0].text)
        else:
            list_property.append(np.nan)
        list_name.append(full_name[0].text)
        list_party.append(party[0].text.split(' ', 1)[-1])
        list_region.append(region[0].text)
        list_elected_times.append(elected_times[0].text)
        list_education.append(education[0].text.replace('\n', '/'))
        list_career.append(career[0].text.replace('\n', '/'))
        list_tel.append(tel[0].text)
        list_email.append(email[0].text)
        list_bill_count.append(bill_count[0].text)
        list_meeting_attend.append(meeting_attend[0].text)
        driver.back()
print("---{}초 걸렸슴둥---".format(time.time()-start_time))


df = pd.DataFrame([list_name, list_party, list_region, list_elected_times, 
                   list_committee, list_education, list_career, list_tel,
                  list_email, list_bill_count, list_committee_attend,
                  list_meeting_attend, list_property]).T
colNames = ['이름' ,'정당', '선거구', '당선횟수', '소속위원회', '학력', '주요경력',
           '연락처', '이메일', '대표발의법안수', '위원회 출석률', '본회의 출석률', '재산']
df.columns = colNames
df

df.to_csv('assembly_members.csv', index=False)

## 사진 크롤링
# 열러라 국회 의원 접속
driver.get('http://watch.peoplepower21.org/?act=&mid=AssemblyMembers&vid=&mode=search&name=&party=&region=&sangim=&gender=&age=&elect_num=')

def download(URL, short_name):
    short_name = short_name[0].text.split(' ',1)[-1]
    fullName = "images/" + short_name + ".jpg"
    urllib.request.urlretrieve(URL, fullName)   

start_time = time.time()

for page in range(1,11):
    if page != 1:
        driver.get('http://watch.peoplepower21.org/?mid=AssemblyMembers&mode=search&party=&region=&sangim=&gender=&elect_num=&page=' + str(page))
    for person in range(2,32):
        short_name = driver.find_elements_by_css_selector('#content > div.col-md-8 > div:nth-child('+ str(person)+')> div > a:nth-child(2) > h4')
        image = driver.find_elements_by_tag_name('#content > div.col-md-8 > div:nth-child(' + str(person)+ ') > div > a:nth-child(1) > img')
        src = image[0].get_attribute('src')
        download(src, short_name)

print("---{}초 걸렸슴둥---".format(time.time()-start_time))

# 2. 데이터 전처리

데이터 부르기

In [9]:
bills = pd.read_csv('bills_24839_all_cleansed.csv')
senators = pd.read_csv('(원본) assembly_memebers_295.csv')

한글이름, 한자이름 column 새로 만들어주기

In [11]:
senators['한글이름'] = senators['이름'].str.split(' ').str[0]
senators['한자이름'] = senators['이름'].str.split(' ').str[1]

함수 정의하기

In [8]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [12]:
def count_발의자(data):
    if type(data) == float:
        result = 0
    else:
        result = [token for token in data.split(" ") if token not in [""]]
        result = len(result)
    return result

In [13]:
bills['발의자 수'] = bills['발의자'].progress_apply(count_발의자)

100%|██████████| 23779/23779 [00:00<00:00, 278874.38it/s]


결측치 제거하기

In [19]:
bills = bills.dropna()

tokenize 진행하기

In [17]:
bills['tokenized'] = bills['법안명_법안내용'].progress_apply(tokenized_mecab)

100%|██████████| 23778/23778 [00:24<00:00, 962.10it/s] 


1차 전처리 후 stopwords 날리기

In [13]:
url= "stopwords ver1.0.txt"

stopwords_file = list(open(url,"r"))
list_stopword = []
for i in stopwords_file:
    list_stopword.append(i[:-1])

In [14]:
def out_stopwords(data, list_stopwords):
    data = data.split(" ")
    list_stopwords = list_stopwords
    data = [token for token in data if token not in list_stopwords]
    return data

stopword 다 제거해주기

In [15]:
bills['cleared'] = bills['tokenized'].progress_apply(lambda x: out_stopwords(x, list_stopwords = list_stopword))

100%|██████████| 23778/23778 [02:11<00:00, 180.94it/s]


list에 있는 값들 꺼내주기

In [16]:
bills['cleared'] = [' '.join(map(str, l)) for l in bills['cleared']]

In [17]:
bills

,bill_id,입안일,법안명,법안내용,발의자,상태,법안명_이름만,법안명_법안내용,발의자 수,tokenized,cleared
0,0,2020-02-24,출입국관리법 일부개정법률안(정성호의원 등 10인),관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류,출입국관리법 일부개정법률안,출입국관리법 일부개정법률안 관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리...,10,출입국관리법 일부개정법률안 관광 등 단기 체류 목적 으로 입국 하 는 외국인 은 출...,출입국관리법 일부개정법률안 관광 단기 체류 목적 입국 외국인 출입국 관리 공무원 입...
1,1,2020-02-24,통신비밀보호법 일부개정법률안(추혜선의원 등 10인),2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류,통신비밀보호법 일부개정법률안,통신비밀보호법 일부개정법률안 2018. 8. 30. 헌법재판소는 국가정보원 인터넷...,11,통신비밀보호법 일부개정법률안 2018 . 8 . 30 . 헌법 재판소 는 국가정보원...,통신비밀보호법 일부개정법률안 헌법 재판소 국가정보원 인터넷 회선 감청 이른바 패킷 ...
2,2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안(신창현의원 등 ...,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안 현행 법령상 ...,10,공익사업 을 위한 토지 등 의 취득 및 보상 에 관한 법률 일부개정법률안 현행 법령...,공익사업 토지 취득 보상 관한 법률 일부개정법률안 현행 법령 상 도시 가스 미공 지...
3,3,2020-02-21,군인사법 일부개정법률안(정부),일과 가정의 양립이 가능한 공직문화 조성을 위하여 첫째 자녀에 대하여 부모가 모두 ...,정부,계류,군인사법 일부개정법률안,군인사법 일부개정법률안 일과 가정의 양립이 가능한 공직문화 조성을 위하여 첫째 자녀...,1,군인사법 일부개정법률안 일 과 가정 의 양립 이 가능 한 공직 문화 조성 을 위하 ...,군인사법 일부개정법률안 가정 양립 가능 공직 문화 조성 자녀 대하 부모 모두 육아휴...
4,4,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안(서영교의원 등 10인),최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류,중소기업진흥에 관한 법률 일부개정법률안,중소기업진흥에 관한 법률 일부개정법률안 최근 코로나바이러스감염증-19(COVID-...,10,중소기업진흥 에 관한 법률 일부개정법률안 최근 코로나 바이러스 감염증 - 19 ( ...,중소기업진흥 관한 법률 일부개정법률안 최근 코로나 바이러스 감염증 COVID 확산 ...
...,...,...,...,...,...,...,...,...,...,...,...
23774,23774,2016-05-30,교육기본법 일부개정법률안(이찬열의원 등 10인),"한편, 최근 저출산 문제의 해결을 위하여 지방자치단체별로 다양한 공공요금의 할인, ...",이찬열(더불어민주당/李燦烈) 김경협(더불어민주당/金炅俠) 백재현(더불어민주당/白在鉉...,계류,교육기본법 일부개정법률안,"교육기본법 일부개정법률안 한편, 최근 저출산 문제의 해결을 위하여 지방자치단체별로 ...",10,"교육기본법 일부개정법률안 한편 , 최근 저출산 문제 의 해결 을 위하 여 지방자치단...",교육기본법 일부개정법률안 한편 최근 저출산 문제 해결 지방자치단체 다양 공공요금 할...
23775,23775,2016-05-30,정부조직법 일부개정법률안(이종배의원 등 11인),통계청에 따르면 우리나라는 2000년에 65세 이상의 노인인구가 전체인구의 7%로...,이종배(새누리당/李鍾培) 권성동(새누리당/權性東) 김재경(새누리당/金在庚) 박덕흠(...,계류,정부조직법 일부개정법률안,정부조직법 일부개정법률안 통계청에 따르면 우리나라는 2000년에 65세 이상의 노...,11,정부 조직법 일부개정법률안 통계청 에 따르 면 우리 나라 는 2000 년 에 65 ...,정부 조직법 일부개정법률안 통계청 따르 우리 나라 이상 노인 인구 전체 인구 고령 ...
23776,23776,2016-05-30,근로기준법 일부개정법률안(이찬열의원 등 11인),지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 노동시장의 모습’을 살펴보...,이찬열(더불어민주당/李燦烈) 김경협(더불어민주당/金炅俠) 김현미(더불어민주당/金賢美...,계류,근로기준법 일부개정법률안,근로기준법 일부개정법률안 지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 ...,13,근로기준법 일부개정법률안 지난 2015 년 12 월 고용노동부 에서 발간 한 ‘ 우...,근로기준법 일부개정법률안 고용노동부 발간 우리 나라 노동 시장 모습 살펴보 현재 우...
23777,23777,2016-05-30,빅데이터의 이용 및 산업진흥 등에 관한 법률안(배덕광의원 등 14인),빅데이터산업은 사물인터넷·클라우드 컴퓨팅 산업 등과 함께 정보통신산업의 성장을 이...,배덕광(새누리당/裵德光) 김세연(새누리당/金世淵) 김정훈(새누리당/金正薰) 김한표(...,계류,빅데이터의 이용 및 산업진흥 등에 관한 법률안,빅데이터의 이용 및 산업진흥 등에 관한 법률안 빅데이터산업은 사물인터넷·클라우드 ...,15,빅데이터 의 이용 및 산업 진흥 등 에 관한 법률안 빅데이터 산업 은 사물 인터넷 ...,빅데이터 이용 산업 진흥 관한 법률안 빅데이터 산업 사물 인터넷 클라우드 컴퓨팅 산...


정부, 위원장이 발의자인 법안 제거하기

In [18]:
df = bills[bills['발의자'] != '위원장']
df = df[df['발의자'] != '정부'] 
df = df[df['발의자 수'] <= 30]

In [19]:
bills = df.copy()

tokenize 완료된 bills들을 json으로 만들기

In [21]:
# 법안과 cleared 내용으로 key, value 매치
bills_json = {}
for idx in list(range(bills.shape[0])):
    temp_dict = {}
    temp_dict['bill_id'] = int(bills['bill_id'].iloc[idx])
    temp_dict['법안명'] = bills['법안명'].iloc[idx]
    temp_dict['cleared_token'] = bills['cleared'].iloc[idx]
    bills_json[str(bills['bill_id'].iloc[idx])] = temp_dict

In [38]:
bills_json['0']

{'bill_id': 0,
 '법안명': '출입국관리법 일부개정법률안(정성호의원 등 10인)',
 'cleared_token': '출입국관리법 일부개정법률안 관광 단기 체류 목적 입국 외국인 출입국 관리 공무원 입국 심사 때 출입국관리법 시행령 따른 입국신고서 출입국 관리 공무원 제출 체류 연락처 허위 입국신고서 제출 경우 제재 방법 입국신고서 내용 정확 성 담보 곤란 함 입국신고서 체류 허위 기재 국내 관광 일정 따라 체류 변경 경우 감염병 환자 나 범법자 국내 소재지 파악 곤란 체류 외국인 효율 관리 한계 음 이상 국내 장기 체류 외국인 외국인 등록 때 체류 입증 자료 지방 출입국 ？ 외국인 관서 장 제출 단기 체류 외국인 체류 입증 자료 제출 제도 단기 체류 외국인 국내 체류 정보 적정 게 파악 체계 공공 안전 확보 고자 함 현행 출입국관리법 시행령 규정 입국신고서 제도 법률 상향 규정 허위 입국신고서 제출 경우 과태료 부과 감염병 예방 관리 관한 법률 따른 위기 경보 발령 또는 국민 보호 공공 안전 러 방지법 따른 테러 경보 발령 법무 부령 경우 단기 체류 외국인 숙박업 외국인 자신 인적사항 제공 도록 숙박업 숙박 외국인 정보 법무부장관 제출 도록 함 안 조제 조제'}

#  3. 단어 벡터 만들기

### 1) 학습할 corpus 만들기

In [22]:
tokenized = ''
for content in bills['cleared'].values:
    tokenized = tokenized + "{} \n".format(content)

In [23]:
corpus = [sent.strip().split(" ") for sent in tokenized.split('\n')]

### 2) word2vec 학습시켜서 단어 벡터 만들기

In [24]:
start = time.time()
embedding_model = Word2Vec(corpus, size=100, window = 3, min_count=10,  
                           workers=5, iter=9000, hs=0, sg=1)
print("time :", time.time() - start)

time : 19074.76328921318


학습한 모델 저장하고

In [25]:
embedding_model.save('word2vec ver21.0.model')

학습한 모델 불러오기

In [4]:
embedding_model = Word2Vec.load('models/word2vec ver21.0.model')

### 3) 단어와 벡터를 dictionary 형태로 묶기

In [5]:
word_dict = {}
for vocab, vector in zip(embedding_model.wv.index2word, embedding_model.wv.vectors):
    word_dict[vocab] = vector

# 4. 법안 벡터 만들기

In [37]:
bills

,bill_id,입안일,법안명,법안내용,발의자,상태,법안명_이름만,법안명_법안내용,발의자 수,tokenized,cleared
0,0,2020-02-24,출입국관리법 일부개정법률안(정성호의원 등 10인),관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류,출입국관리법 일부개정법률안,출입국관리법 일부개정법률안 관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리...,10,출입국관리법 일부개정법률안 관광 등 단기 체류 목적 으로 입국 하 는 외국인 은 출...,출입국관리법 일부개정법률안 관광 단기 체류 목적 입국 외국인 출입국 관리 공무원 입...
1,1,2020-02-24,통신비밀보호법 일부개정법률안(추혜선의원 등 10인),2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류,통신비밀보호법 일부개정법률안,통신비밀보호법 일부개정법률안 2018. 8. 30. 헌법재판소는 국가정보원 인터넷...,11,통신비밀보호법 일부개정법률안 2018 . 8 . 30 . 헌법 재판소 는 국가정보원...,통신비밀보호법 일부개정법률안 헌법 재판소 국가정보원 인터넷 회선 감청 이른바 패킷 ...
2,2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안(신창현의원 등 ...,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안 현행 법령상 ...,10,공익사업 을 위한 토지 등 의 취득 및 보상 에 관한 법률 일부개정법률안 현행 법령...,공익사업 토지 취득 보상 관한 법률 일부개정법률안 현행 법령 상 도시 가스 미공 지...
4,4,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안(서영교의원 등 10인),최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류,중소기업진흥에 관한 법률 일부개정법률안,중소기업진흥에 관한 법률 일부개정법률안 최근 코로나바이러스감염증-19(COVID-...,10,중소기업진흥 에 관한 법률 일부개정법률안 최근 코로나 바이러스 감염증 - 19 ( ...,중소기업진흥 관한 법률 일부개정법률안 최근 코로나 바이러스 감염증 COVID 확산 ...
5,5,2020-02-19,주거급여법 일부개정법률안(함진규의원 등 10인),현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,함진규(자유한국당/咸珍圭) 강석진(자유한국당/姜錫振) 김성원(자유한국당/金成願) 김...,계류,주거급여법 일부개정법률안,주거급여법 일부개정법률안 현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주...,10,주거급여법 일부개정법률안 현행법 은 「 국민 기초 생활 보장 법 」 관련 규정 을 ...,주거급여법 일부개정법률안 현행법 국민 기초 생활 보장 법 관련 규정 준용 주거급여 ...
...,...,...,...,...,...,...,...,...,...,...,...
23774,23774,2016-05-30,교육기본법 일부개정법률안(이찬열의원 등 10인),"한편, 최근 저출산 문제의 해결을 위하여 지방자치단체별로 다양한 공공요금의 할인, ...",이찬열(더불어민주당/李燦烈) 김경협(더불어민주당/金炅俠) 백재현(더불어민주당/白在鉉...,계류,교육기본법 일부개정법률안,"교육기본법 일부개정법률안 한편, 최근 저출산 문제의 해결을 위하여 지방자치단체별로 ...",10,"교육기본법 일부개정법률안 한편 , 최근 저출산 문제 의 해결 을 위하 여 지방자치단...",교육기본법 일부개정법률안 한편 최근 저출산 문제 해결 지방자치단체 다양 공공요금 할...
23775,23775,2016-05-30,정부조직법 일부개정법률안(이종배의원 등 11인),통계청에 따르면 우리나라는 2000년에 65세 이상의 노인인구가 전체인구의 7%로...,이종배(새누리당/李鍾培) 권성동(새누리당/權性東) 김재경(새누리당/金在庚) 박덕흠(...,계류,정부조직법 일부개정법률안,정부조직법 일부개정법률안 통계청에 따르면 우리나라는 2000년에 65세 이상의 노...,11,정부 조직법 일부개정법률안 통계청 에 따르 면 우리 나라 는 2000 년 에 65 ...,정부 조직법 일부개정법률안 통계청 따르 우리 나라 이상 노인 인구 전체 인구 고령 ...
23776,23776,2016-05-30,근로기준법 일부개정법률안(이찬열의원 등 11인),지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 노동시장의 모습’을 살펴보...,이찬열(더불어민주당/李燦烈) 김경협(더불어민주당/金炅俠) 김현미(더불어민주당/金賢美...,계류,근로기준법 일부개정법률안,근로기준법 일부개정법률안 지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 ...,13,근로기준법 일부개정법률안 지난 2015 년 12 월 고용노동부 에서 발간 한 ‘ 우...,근로기준법 일부개정법률안 고용노동부 발간 우리 나라 노동 시장 모습 살펴보 현재 우...
23777,23777,2016-05-30,빅데이터의 이용 및 산업진흥 등에 관한 법률안(배덕광의원 등 14인),빅데이터산업은 사물인터넷·클라우드 컴퓨팅 산업 등과 함께 정보통신산업의 성장을 이...,배덕광(새누리당/裵德光) 김세연(새누리당/金世淵) 김정훈(새누리당/金正薰) 김한표(...,계류,빅데이터의 이용 및 산업진흥 등에 관한 법률안,빅데이터의 이용 및 산업진흥 등에 관한 법률안 빅데이터산업은 사물인터넷·클라우드 ...,15,빅데이터 의 이용 및 산업 진흥 등 에 관한 법률안 빅데이터 산업 은 사물 인터넷 ...,빅데이터 이용 산업 진흥 관한 법률안 빅데이터 산업 사물 인터넷 클라우드 컴퓨팅 산...


In [38]:
bills_json['0']

{'bill_id': 0,
 '법안명': '출입국관리법 일부개정법률안(정성호의원 등 10인)',
 'cleared_token': '출입국관리법 일부개정법률안 관광 단기 체류 목적 입국 외국인 출입국 관리 공무원 입국 심사 때 출입국관리법 시행령 따른 입국신고서 출입국 관리 공무원 제출 체류 연락처 허위 입국신고서 제출 경우 제재 방법 입국신고서 내용 정확 성 담보 곤란 함 입국신고서 체류 허위 기재 국내 관광 일정 따라 체류 변경 경우 감염병 환자 나 범법자 국내 소재지 파악 곤란 체류 외국인 효율 관리 한계 음 이상 국내 장기 체류 외국인 외국인 등록 때 체류 입증 자료 지방 출입국 ？ 외국인 관서 장 제출 단기 체류 외국인 체류 입증 자료 제출 제도 단기 체류 외국인 국내 체류 정보 적정 게 파악 체계 공공 안전 확보 고자 함 현행 출입국관리법 시행령 규정 입국신고서 제도 법률 상향 규정 허위 입국신고서 제출 경우 과태료 부과 감염병 예방 관리 관한 법률 따른 위기 경보 발령 또는 국민 보호 공공 안전 러 방지법 따른 테러 경보 발령 법무 부령 경우 단기 체류 외국인 숙박업 외국인 자신 인적사항 제공 도록 숙박업 숙박 외국인 정보 법무부장관 제출 도록 함 안 조제 조제'}

법안 속에 있는 모든 단어의 벡터를 더해서 법안 벡터 생성해서 json으로 저장해주기

In [39]:
for idx in tqdm(bills_json):
    list_vector =[]
    for word in bills_json[idx]['cleared_token'].split():
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    bills_json[idx]['vector'] = np.sum(list_vector, axis=0).tolist()

100%|██████████| 21066/21066 [00:02<00:00, 7755.14it/s]


저장하기

In [40]:
with open('bills ver21.0.json', 'w', encoding='utf-8') as make_file:
    json.dump(bills_json, make_file, indent="\t")

# 5. 국회의원 벡터 만들기

국회의원 json 만들기

In [44]:
senators_json = {}
for idx in list(range(senators.shape[0])):
    temp_dict = {}
    temp_dict['한글이름'] = senators['한글이름'].iloc[idx]
    temp_dict['한자이름'] = senators['한자이름'].iloc[idx]
    temp_dict['정당'] = senators['정당'].iloc[idx]
    senators_json[senators['이름'].iloc[idx]] = temp_dict

In [45]:
senators_json

{'강길부 姜吉夫': {'한글이름': '강길부', '한자이름': '姜吉夫', '정당': '무소속'},
 '강병원 姜炳遠': {'한글이름': '강병원', '한자이름': '姜炳遠', '정당': '더불어민주당'},
 '강석진 姜錫振': {'한글이름': '강석진', '한자이름': '姜錫振', '정당': '미래통합당'},
 '강석호 姜碩鎬': {'한글이름': '강석호', '한자이름': '姜碩鎬', '정당': '미래통합당'},
 '강창일 姜昌一': {'한글이름': '강창일', '한자이름': '姜昌一', '정당': '더불어민주당'},
 '강효상 姜孝祥': {'한글이름': '강효상', '한자이름': '姜孝祥', '정당': '미래통합당'},
 '강훈식 姜勳植': {'한글이름': '강훈식', '한자이름': '姜勳植', '정당': '더불어민주당'},
 '경대수 慶大秀': {'한글이름': '경대수', '한자이름': '慶大秀', '정당': '미래통합당'},
 '고용진 高榕禛': {'한글이름': '고용진', '한자이름': '高榕禛', '정당': '더불어민주당'},
 '곽대훈 郭大勳': {'한글이름': '곽대훈', '한자이름': '郭大勳', '정당': '미래통합당'},
 '곽상도 郭尙道': {'한글이름': '곽상도', '한자이름': '郭尙道', '정당': '미래통합당'},
 '권미혁 權美赫': {'한글이름': '권미혁', '한자이름': '權美赫', '정당': '더불어민주당'},
 '권성동 權性東': {'한글이름': '권성동', '한자이름': '權性東', '정당': '미래통합당'},
 '권은희 權垠希': {'한글이름': '권은희', '한자이름': '權垠希', '정당': '국민의당'},
 '권칠승 權七勝': {'한글이름': '권칠승', '한자이름': '權七勝', '정당': '더불어민주당'},
 '금태섭 琴泰燮': {'한글이름': '금태섭', '한자이름': '琴泰燮', '정당': '더불어민주당'},
 '기동민 奇東旻': {'한글이름': '기동민', '한자이름': '奇東旻', '정당': '더불

발의에 참여한 법안의 벡터를 모두 더해서 국회의원 벡터 만들기

In [46]:
for idx in tqdm(list(range(senators.shape[0]))):
    full_name = senators['이름'].iloc[idx]
    chinese_name = senators['한자이름'].iloc[idx]
    korean_name = senators['한글이름'].iloc[idx]
    if korean_name == '김성태':
        continue
    participated_bills = list(bills[bills['발의자'].str.contains(korean_name)]['bill_id'].values)
    bill_vector_list = []
    for bill_id in participated_bills:
        bill_vector_list.append(np.array(bills_json[str(bill_id)]['vector']))
    senators_json[full_name]['vector'] = (np.sum(bill_vector_list, axis=0)/len(bill_vector_list)).tolist()

100%|██████████| 295/295 [00:06<00:00, 46.78it/s]


김성태 의원이 2명이어서 이 의원은 따로 만들어줬음

In [47]:
# 이름 겹치는 김성태 의원만 다시 구해주기
for name in tqdm(['金成泰','金聖泰']):
    full_name = ['김성태 金成泰', '김성태 金聖泰']
    participated_bills = list(bills[bills['발의자'].str.contains(name)]['bill_id'].values)
    bill_vector_list = []
    for bill_id in participated_bills:
        bill_vector_list.append(np.array(bills_json[str(bill_id)]['vector']))
    for name1 in full_name:
        senators_json[name1]['vector'] = (np.sum(bill_vector_list, axis=0)/len(bill_vector_list)).tolist()

100%|██████████| 2/2 [00:00<00:00, 50.72it/s]


완성된 국회의원 벡터 저장하기

In [49]:
with open('senators ver21.0.json', 'w', encoding='utf-8') as make_file:
    json.dump(senators_json, make_file, indent="\t")

# 6. 의원, 단어, 법안 시각화하기

### 1) 의원 시각화

In [3]:
with open('senators ver21.0.json', 'r') as f:
    senators_json = json.load(f)

In [4]:
list_vector = []
list_senator = []
for senator in senators_json:
    vector_array = np.array(senators_json[senator]['vector'])
    list_senator.append(senators_json[senator]['한글이름'])
    list_vector.append(vector_array)

In [28]:
tsne = TSNE(n_components=2)

In [29]:
senator_tsne = tsne.fit_transform(vector_array[:,:])

In [30]:
df_senator = pd.DataFrame(senator_tsne, index=list_senator, columns=['x', 'y'])

In [31]:
df_senator_with_party = df_senator.reset_index().merge(senators[['한글이름', '정당']],how='left', left_on='index', right_on='한글이름')
del df_senator_with_party['한글이름']

In [32]:
df_senator_with_party

,index,x,y,정당
0,강길부,-7.843739,9.154598,무소속
1,강병원,-11.518075,6.042161,더불어민주당
2,강석진,5.722982,-15.073046,미래통합당
3,강석호,8.049164,6.404850,미래통합당
4,강창일,3.764485,-7.717409,더불어민주당
...,...,...,...,...
292,홍일표,-12.426230,11.993715,미래통합당
293,홍철호,1.022686,5.523757,미래통합당
294,황주홍,10.712513,-13.691450,민생당
295,황희,-0.078042,6.136267,더불어민주당


In [33]:
df_senator_with_party.to_csv("senator map ver.21.0.csv", index=False)

In [74]:
dict_senator_with_party = dict(x=df_senator_with_party.x.tolist(), y=df_senator_with_party.y.tolist(), text=df_senator_with_party['index'].tolist() )
senator_data = ColumnDataSource(dict_senator_with_party)
parties = ['더불어민주당', '미래통합당', '국민의당', '민생당','미래한국당','정의당','우리공화당', '민중당', '무소속']
colors = ['blue', 'red', 'green', 'blue', 'pink', 'yellow', 'black', '#ff7f0e', 'white']

senator_with_party = figure(title='Senator with party ver21.0',
                            height=1500, width=1500, active_scroll='wheel_zoom')

senator_with_party.scatter(x='x', y='y', alpha=0.5, size=10, source=df_senator_with_party,
    color = factor_cmap('정당', colors, parties), legend_field='정당')

glyph = Text(x="x", y="y", text="text", angle=0, text_color="black")

senator_with_party.add_glyph(senator_data, glyph)

senator_with_party.title.text_font_size = value('16pt')
senator_with_party.xaxis.visible = False
senator_with_party.yaxis.visible = False
senator_with_party.grid.grid_line_color = None
senator_with_party.outline_line_color = None
show(senator_with_party)

저장하기

In [ ]:
output_file("senator map ver21.0.html")
save(senator_with_party)

### 2) 단어 시각화

In [78]:
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
model = embedding_model
vocab = list(model.wv.vocab)
X = model[vocab]

tsne = TSNE(n_components=2)

# 100개의 단어에 대해서만 시각화
X_tsne = tsne.fit_transform(X[:,:])
# X_tsne = tsne.fit_transform(X)

<ipython-input-78-849625d854ce>:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  X = model[vocab]


In [79]:
df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y'])
df.shape

(9680, 2)

In [80]:
df

,x,y
출입국관리법,38.944962,14.131817
일부개정법률안,40.737644,-35.385429
관광,-0.123371,-29.369726
단기,1.706792,-5.512823
체류,38.327591,16.421406
...,...,...
무궁화,55.669960,10.733881
도굴,-6.300141,48.844627
비정규군,43.955055,21.809519
나라꽃,55.671104,10.725904


In [82]:
tsne_df = df.copy()

In [83]:
tsne_df

,x,y
출입국관리법,38.944962,14.131817
일부개정법률안,40.737644,-35.385429
관광,-0.123371,-29.369726
단기,1.706792,-5.512823
체류,38.327591,16.421406
...,...,...
무궁화,55.669960,10.733881
도굴,-6.300141,48.844627
비정규군,43.955055,21.809519
나라꽃,55.671104,10.725904


In [84]:
dict_tsne = dict(x=tsne_df.x.tolist(), y=tsne_df.y.tolist(), text=tsne_df.index.tolist() )

In [85]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(dict_tsne)

# create the plot and configure it
word_plot = figure(title='Word Embeddings ver21.0',
                   plot_width = 1500,
                   plot_height = 1500,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
word_plot.add_tools( HoverTool(tooltips = '@word') )

word_plot.circle('x', 'y', source=plot_data,
                 color='green', line_alpha=0.4, fill_alpha=0.1,
                 size=5, hover_line_color='orange')
glyph = Text(x="x", y="y", text="text", angle=0, text_color="black")
word_plot.add_glyph(plot_data, glyph)
# adjust visual elements of the plot
word_plot.title.text_font_size = value('16pt')
word_plot.xaxis.visible = False
word_plot.yaxis.visible = False
word_plot.grid.grid_line_color = None
word_plot.outline_line_color = None

# show time!
show(word_plot);

In [ ]:
output_file("word map ver21.0.html")
save(word_plot)

### 3) 법안 시각화

In [4]:
with open('bills ver21.0.json', 'r') as f:
    bills_json = json.load(f)

In [5]:
list_vector = []
list_bills = []
for bill in bills_json:
    vector_array = np.array(bills_json[bill]['vector'])
    list_bills.append(bills_json[bill]['bill_id'])
    list_vector.append(vector_array)

In [6]:
bill_vector_array = np.array(list_vector)

In [7]:
bills_tsne = tsne.fit_transform(bill_vector_array[:,:])

NameError: name 'tsne' is not defined

In [91]:
df_bills = pd.DataFrame(bills_tsne, index=list_bills, columns=['x', 'y'])
df_bills

,x,y
0,32.335327,0.049392
1,-21.040977,62.774551
2,-1.656494,-54.839699
4,-25.325911,-9.509981
5,0.232976,6.142670
...,...,...
23774,0.924670,8.904943
23775,5.483482,4.365985
23776,-0.117456,34.649994
23777,-44.104237,44.611271


In [92]:
dict_bills = dict(x=df_bills.x.tolist(), y=df_bills.y.tolist(), text=df_bills.index.tolist() )

In [93]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(dict_bills)

# create the plot and configure it
bills_plot = figure(title='Bills Embeddings ver21.0',
                   plot_width = 1500,
                   plot_height = 1500,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
bills_plot.add_tools( HoverTool(tooltips = '@word') )

bills_plot.circle('x', 'y', source=plot_data,
                 color='green', line_alpha=0.4, fill_alpha=0.1,
                 size=5, hover_line_color='orange')
glyph = Text(x="x", y="y", text="text", angle=0, text_color="black")
bills_plot.add_glyph(plot_data, glyph)
# adjust visual elements of the plot
bills_plot.title.text_font_size = value('16pt')
bills_plot.xaxis.visible = False
bills_plot.yaxis.visible = False
bills_plot.grid.grid_line_color = None
bills_plot.outline_line_color = None

# show time!
show(bills_plot);

In [ ]:
output_file("bills map ver21.0.html")
save(bills_plot)

# 7. 워드 클라우드 만들어보기

파일 부르기

In [ ]:
with open('senator with cluster ver1.0.json', 'r') as f:
    json_senator = json.load(f)

stopwords 준비하기

In [ ]:
url= "stopwords ver1.0.txt"

stopwords_file = list(open(url,"r"))
list_stopword = []
for i in stopwords_file:
    list_stopword.append(i[:-1])

mask_array = np.array(Image.open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_열려라 국회/images/대한민국.jpeg"))
stopwords_ = ['일부개정법률안','현행법','의안번호','려는','관한','조제','신설', '도록', '으로써','주요','내용','또는'
              '이하','벌금','천만','징역','또는','참고사항','의결','아니하','법률안 의결', '경우', '조정','경우','경우조정',
              '전제','법률안','따른','이하','어야','그러나','관련','대해서','부터','으며','대해','해당','아니','추가',
              '국가', '대한','민국','위원회','따라','이상','고자','지원','규정','필요','사업',
              '대하', '기준', '그런데','또한', '해야', '으로서']
stopwords_.extend(list_stopword)

word cloud 만들기

In [ ]:
for num in tqdm(list(range(0,13))):
    cluster_num = num
    print("cluster",cluster_num,"시작","*"*50 )

    # cluster 내 발의 법안들 모아주기
    list_collect_ids = []
    for senator in json_senator['cluster {}'.format(cluster_num)]:
        list_collect_ids.extend(json_senator['cluster {}'.format(cluster_num)][senator]['total_bill_ids'])
    print("cluster 내 발의법안 수: ", len(set(list_collect_ids)))

    # 공통 발의법안들 모아주기
    word_bags_cluster = ''
    for i in Counter(list_collect_ids).most_common(round(len(set(list_collect_ids))/100)):
        corpus = bills[bills['bill_id']==i[0]]['cleared_token'].iloc[0]
        word_bags_cluster = word_bags_cluster + corpus

    # wordcloud 만들기
    wc = WordCloud(background_color = "ivory",
                   font_path = "/Library/Fonts/NanumSquareRegular.ttf",
                   colormap = "Reds",
                   collocations = True, 
                   stopwords = stopwords_,
                   mask = mask_array)

    # WordCloud 출력하기
    wc.generate(word_bags_cluster)
    wc.to_image()
    
    # wordcloud 저장하기
    wc.to_file('cluster_{} ver7.0 1%.png'.format(cluster_num))

# 8. 사용자 Input 받아서 국회의원 추천하기

모델 불러오기

In [2]:
embedding_model = Word2Vec.load('word2vec ver21.0.model')

In [3]:
word_dict = {}
for vocab, vector in zip(embedding_model.wv.index2word, embedding_model.wv.vectors):
    word_dict[vocab] = vector

국회의원 데이터 불러오기

In [4]:
with open('senators ver21.0.json', 'r') as f:
    senators_json = json.load(f)

법안 데이터 불러오기

In [5]:
bills = pd.read_csv('bills_24839_all_cleansed.csv')

In [6]:
bills = bills.dropna()

함수모음

In [7]:
def search(user_input):
    temp_list = []
    for word in word_dict:
        if user_input in word:
            temp_list.append(word)
    return temp_list

def similarWord(model, text):
    return model.most_similar(positive=[text], topn=30)

def similar_senators(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    return rating[:5]

def show_senators_bills_contents(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_ids = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['bill_id'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_ids.extend(semi_result)
        list_temp_keyword_bills_ids = list(set(list_temp_keyword_bills_ids))
        shuffle(list_temp_keyword_bills_ids)
        list_bills_title = df_bills[df_bills['bill_id'].isin(list_temp_keyword_bills_ids[:3])]['법안명'].values.tolist()
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

def show_senators_bills_titles(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_titles = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['법안명'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_titles.extend(semi_result)
        list_temp_keyword_bills_titles = list(set(list_temp_keyword_bills_titles))
        shuffle(list_temp_keyword_bills_titles)
        print(list_temp_keyword_bills_titles)
        list_bills_titlle = df_bills.loc(list_temp_keyword_bills_titles[:3])
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

### 1) 첫 번째 시연: 20대 취업준비중인 청년

In [8]:
search('청년')

['청년', '청년정책', '청년층', '청년실업률', '청년기본소득']

In [9]:
search('취업')

['취업', '재취업', '취업지원', '취업규칙', '취업자']

In [10]:
similarWord(embedding_model, '청년')

<ipython-input-7-a19bf89c2456>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return model.most_similar(positive=[text], topn=30)


[('미취', 0.7219280004501343),
 ('고용촉진', 0.6167274713516235),
 ('실업', 0.5831162929534912),
 ('청년정책', 0.5714389681816101),
 ('일자리', 0.5657309293746948),
 ('창업', 0.5624954700469971),
 ('청년층', 0.5527931451797485),
 ('비정규직', 0.5297333002090454),
 ('고용', 0.5255996584892273),
 ('구직', 0.5193198323249817),
 ('취업', 0.5191558599472046),
 ('정규직', 0.4891175627708435),
 ('중소기업', 0.4889249801635742),
 ('저출산', 0.4814630150794983),
 ('사회상', 0.4796830415725708),
 ('실업자', 0.47934767603874207),
 ('채움', 0.47375449538230896),
 ('신혼', 0.4728226065635681),
 ('지방공기업', 0.4708821475505829),
 ('내일', 0.4685583710670471),
 ('주거지원', 0.4595612585544586),
 ('고령사회', 0.4569365680217743),
 ('인재', 0.4538780748844147),
 ('창업자', 0.4537651538848877),
 ('직업', 0.45215409994125366),
 ('농업인', 0.44717732071876526),
 ('창출', 0.44420331716537476),
 ('저성장', 0.44353532791137695),
 ('예술인', 0.4422413110733032),
 ('고용창출', 0.4418375790119171)]

In [11]:
similarWord(embedding_model, '취업')

<ipython-input-7-a19bf89c2456>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return model.most_similar(positive=[text], topn=30)


[('재취업', 0.6429386138916016),
 ('직업훈련', 0.5414436459541321),
 ('구직자', 0.5408217310905457),
 ('미취', 0.5377607345581055),
 ('학자금', 0.5357702970504761),
 ('구직', 0.5332841277122498),
 ('퇴직', 0.5293415784835815),
 ('직업', 0.5228592753410339),
 ('청년', 0.5191558599472046),
 ('취업지원', 0.5092296600341797),
 ('실업', 0.49163392186164856),
 ('체류자격', 0.4875286817550659),
 ('채용', 0.4841924011707306),
 ('학업', 0.4793766736984253),
 ('경력', 0.47280240058898926),
 ('결혼', 0.4702881872653961),
 ('복직', 0.4679752588272095),
 ('재직', 0.4602130651473999),
 ('창업', 0.4599595367908478),
 ('취업자', 0.45899713039398193),
 ('근무', 0.45837974548339844),
 ('보미', 0.4528379440307617),
 ('학력', 0.4513038396835327),
 ('복귀', 0.4489874243736267),
 ('겸직', 0.4477297067642212),
 ('재차', 0.4476053714752197),
 ('진학', 0.447547972202301),
 ('고용창출', 0.44513899087905884),
 ('실업자', 0.4418206214904785),
 ('입사', 0.43755245208740234)]

In [12]:
user_input=['취업', '비정규직', '근로조건','청년실업률','취업지원']

In [13]:
list_senator = similar_senators(user_input, senators_json, word_dict)
list_senator

['김현미 더불어민주당', '심상정 정의당', '이정미 정의당', '김부겸 더불어민주당', '김태년 더불어민주당']

In [14]:
show_senators_bills_contents(list_senator, bills)

{'김현미 더불어민주당': ['지방자치단체를 당사자로 하는 계약에 관한 법률 일부개정법률안(김현미의원 등 16인)',
  '취업 후 학자금 상환 특별법 일부개정법률안(김민기의원 등 23인)',
  '고용보험법 일부개정법률안(박남춘의원 등 11인)'],
 '심상정 정의당': ['청년사회상속법안(심상정의원 등 12인)',
  '청년고용촉진 특별법 일부개정법률안(이정미의원 등 11인)',
  '최고임금법안(심상정의원 등 10인)'],
 '이정미 정의당': ['고용정책 기본법 일부개정법률안(이정미의원 등 11인)',
  '가사근로자의 고용개선 등에 대한 법률안(이정미의원 등 10인)',
  '고용보험법 일부개정법률안(이정미의원 등 10인)'],
 '김부겸 더불어민주당': ['근로기준법 일부개정법률안(김경협의원 등 12인)',
  '국가재정법 일부개정법률안(정세균의원 등 108인)',
  '근로기준법 일부개정법률안(이용득의원 등 22인)'],
 '김태년 더불어민주당': ['공공기관의 운영에 관한 법률 일부개정법률안(박광온의원 등 10인)',
  '청년세법안(정세균의원 등 108인)',
  '조세특례제한법 일부개정법률안(박광온의원 등 10인)']}

### 2) 두 번째 시연: 장애인 자녀를 키우고 있는 부모

In [15]:
search('장애인')

['장애인', '장애인복지법', '시각장애인', '장애인복지시설', '장애인차별금지', '장애인기업']

In [16]:
search('복지')

['복지',
 '보건복지부장관',
 '복지시설',
 '보건복지부',
 '복지서비스',
 '장애인복지법',
 '사회복지사',
 '장애인복지시설',
 '복지향상',
 '사회복지사업법',
 '복지지원',
 '근로복지공단',
 '복지급여',
 '사회복지',
 '군인복지기금',
 '복지수요',
 '한국보훈복지의료공단법',
 '복지사각지대']

In [17]:
similarWord(embedding_model, '장애인')

<ipython-input-7-a19bf89c2456>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return model.most_similar(positive=[text], topn=30)


[('장애', 0.7896611094474792),
 ('청각', 0.5965456962585449),
 ('고령자', 0.5953530073165894),
 ('애인', 0.5710546970367432),
 ('노약자', 0.5699296593666077),
 ('노인', 0.5614126920700073),
 ('중증', 0.552778959274292),
 ('시각장애인', 0.5402193069458008),
 ('임산부', 0.5340600609779358),
 ('달장', 0.5313823819160461),
 ('장애인복지법', 0.5272414684295654),
 ('시청각', 0.5255222320556641),
 ('관람', 0.5157126188278198),
 ('계층', 0.5129716992378235),
 ('안내', 0.5102976560592651),
 ('아동', 0.5057249069213867),
 ('약자', 0.5045624375343323),
 ('예술인', 0.5011786222457886),
 ('점자', 0.4989457130432129),
 ('재활', 0.49142831563949585),
 ('보조인', 0.49137306213378906),
 ('창작', 0.4912981390953064),
 ('휠체어', 0.4860910177230835),
 ('복지지원', 0.48427218198776245),
 ('장애인복지시설', 0.4825817346572876),
 ('투표소', 0.48202720284461975),
 ('참정권', 0.4745379686355591),
 ('혼자', 0.47376954555511475),
 ('배려', 0.4731495976448059),
 ('활법', 0.4722161889076233)]

In [18]:
similarWord(embedding_model, '복지서비스')

<ipython-input-7-a19bf89c2456>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return model.most_similar(positive=[text], topn=30)


[('정신질환자', 0.5962392091751099),
 ('자립생활', 0.5725449919700623),
 ('휴양', 0.5600941777229309),
 ('정신건강증진', 0.5438527464866638),
 ('서비스', 0.5054366588592529),
 ('아이돌봄', 0.5025054216384888),
 ('귀촌', 0.49260568618774414),
 ('아이돌봄서비스', 0.49185529351234436),
 ('사회복지사', 0.49120980501174927),
 ('재활', 0.489960253238678),
 ('임종', 0.48597246408462524),
 ('급성', 0.4837753176689148),
 ('보육', 0.48339173197746277),
 ('복지', 0.480435311794281),
 ('장애인복지시설', 0.47521939873695374),
 ('사회서비스', 0.47422102093696594),
 ('돌봄', 0.4699277877807617),
 ('환승', 0.46328356862068176),
 ('묘지', 0.462726891040802),
 ('치매', 0.46152329444885254),
 ('자립', 0.4607985019683838),
 ('의증', 0.4602917432785034),
 ('어촌', 0.46020394563674927),
 ('사회복귀', 0.45776695013046265),
 ('경로당', 0.4563027024269104),
 ('퇴소', 0.45537376403808594),
 ('귀농', 0.4551678001880646),
 ('탈시설', 0.4545254707336426),
 ('제대', 0.45409685373306274),
 ('산림문화', 0.45274657011032104)]

In [19]:
user_input=['복지서비스','장애인', '특수교육','장애인복지시설']

In [20]:
list_senator = similar_senators(user_input, senators_json, word_dict)
list_senator

['여영국 정의당', '오영훈 더불어민주당', '정춘숙 더불어민주당', '유승민 미래통합당', '나경원 미래통합당']

In [21]:
show_senators_bills_contents(list_senator, bills)

{'여영국 정의당': ['장애인활동 지원에 관한 법률 일부개정법률안(윤소하의원 등 12인)',
  '국회법 일부개정법률안(추혜선의원 등 14인)',
  '청년고용촉진 특별법 일부개정법률안(이정미의원 등 11인)'],
 '오영훈 더불어민주당': ['정신건강증진 및 정신질환자 복지서비스 지원에 관한 법률 일부개정법률안(신창현의원 등 10인)',
  '공직선거법 일부개정법률안(조승래의원 등 15인)',
  '한국관광공사법 일부개정법률안(손혜원의원 등 13인)'],
 '정춘숙 더불어민주당': ['장애인 건강권 및 의료접근성 보장에 관한 법률 일부개정법률안(권미혁의원 등 11인)',
  '식품위생법 일부개정법률안(양승조의원 등 10인)',
  '화장품법 일부개정법률안(양승조의원 등 10인)'],
 '유승민 미래통합당': ['최저임금법 일부개정법률안(박인숙의원 등 10인)',
  '국가유공자 등 예우 및 지원에 관한 법률 일부개정법률안(지상욱의원 등 11인)',
  '영화 및 비디오물의 진흥에 관한 법률 일부개정법률안(김세연의원 등 10인)'],
 '나경원 미래통합당': ['교통약자의 이동편의 증진법 일부개정법률안(나경원의원 등 10인)',
  '방송법 일부개정법률안(강효상의원 등 11인)',
  '국민체육진흥법 일부개정법률안(나경원의원 등 11인)']}